# Evaluation on PadChest

In [ ]:
import sys
import os
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score
from hydra import compose, initialize
from omegaconf import DictConfig

sys.path.append("/vol/biomedic3/mb121/causal-contrastive")
from classification.classification_module import ClassificationModule
from data_handling.xray import PadChestDataModule
from evaluation.helper_functions import run_inference
os.chdir("/vol/biomedic3/mb121/causal-contrastive/evaluation")

In [ ]:
# Mapping from human readable run name to Weights&Biases run_id. 

# Human readable name should be in format:
# for finetuning:
# {simclr/simclrcf/simclrcfaug}-{train_prop}-{seed}
# for linear probing
# {simclr/simclrcf/simclrcfaug}head-{train_prop}-{seed}

model_dict_normal = {
    "supervised-0.1-11": "8vouetwb",
    "supervised-0.1-22": "96rhabqe",
    "supervised-0.1-33": "fyismsp9",
    "supervised-1.0-22": "4oaaikt6",
    "supervised-1.0-33": "n8h9oapi",
    "supervised-1.0-11": "ghopk3ju",
    "simclr-0.1-22": "n4q4a33f",
    "simclr-0.1-33": "3z4co0f5",
    "simclr-0.1-11": "u2hzkzi8",
    "simclr-1.0-22": "4pe40g5x",
    "simclr-1.0-33": "fph8u80n",
    "simclr-1.0-11": "hqvqp4mh",
    "simclrhead-1.0-33": "bctalde0",
    "simclrhead-0.1-33": "ubv6eqfz",
    "simclrhead-0.1-22": "wi03qw05",
    "simclrhead-1.0-22": "g92xolyv",
    "simclrhead-0.1-11": "lfhrzwlc",
    "simclrhead-1.0-11": "tc5riibq",
    "simclrcfaug-0.1-33": "zqv28q1c",
    "simclrcfaug-1.0-33": "hu8wg6s4",
    "simclrcfaug-0.1-22": "8xf5b6ts",
    "simclrcfaug-1.0-22": "e4k7i59o",
    "simclrcfaug-0.1-11": "n7rp8bvc",
    "simclrcfaug-1.0-11": "ghtxoeph",
    "simclrcf-0.1-11": "597enhyw",
    "simclrcf-0.1-22": "erjrh75z",
    "simclrcf-1.0-22": "axb7nhet",
    "simclrcf-1.0-33": "gcn4pmb5",
    "simclrcf-1.0-11": "bjurudm1",
    "simclrcf-0.1-33": "givxkg9r",
    "simclrcfhead-0.1-22": "2cfdu2ru",
    "simclrcfhead-1.0-11": "aev41p0o",
    "simclrcfhead-0.1-11": "ql05brkt",
    "simclrcfhead-0.25-11": "p57dgkwp",
    "simclrcfhead-0.05-11": "go6laxdr",
    "simclrcfhead-0.05-22": "vp98mhg4",
    "simclrcfhead-0.25-22": "6wjmiqcf",
    "simclrcfhead-0.05-33": "zwpn5i97",
    "simclrcfhead-0.1-33": "7ksiuau7",
    "simclrcfhead-0.25-33": "xjhdufsk",
    "simclrcfhead-1.0-33": "qoqqvcjk",
    "simclrcfhead-1.0-22": "9o431mp5",
    "simclrcf-0.05-11": "yuw1hej0",
    "simclrcf-0.25-11": "zzr4utap",
    "simclrcf-0.05-22": "46orb30r",
    "simclrcf-0.25-22": "sg6h109z",
    "simclrcf-0.05-33": "ues31163",
    "simclrcf-0.25-33": "x20kayvz",
    "simclr-0.05-11": "xuat4sf9",
    "simclr-0.25-11": "03r51wrk",
    "simclr-0.05-22": "noeakj31",
    "simclr-0.25-22": "9k2na4p0",
    "simclr-0.05-33": "qoum861b",
    "simclr-0.25-33": "9ofn816x",
    "simclrcfaug-0.05-11": "84ijd323",
    "simclrcfaug-0.25-11": "2mveqyje",
    "simclrcfaug-0.05-22": "9sg7nlcn",
    "simclrcfaug-0.25-22": "7r316b1j",
    "simclrcfaug-0.05-33": "edk3wfn3",
    "simclrcfaug-0.25-33": "yuwmsfoz",
    "simclrcfaughead-0.05-22": "0sqtorl6",
    "simclrcfaughead-0.05-11": "8m2e86xr",
    "simclrcfaughead-0.05-33": "arzhs7kv",
    "simclrcfaughead-0.25-33": "wpz8ltrm",
    "simclrcfaughead-0.25-22": "pai5bnm4",
    "simclrcfaughead-0.25-11": "x0pmqpy9",
    "simclrhead-0.1-33": "pqr1shrw",
    "simclrcfaughead-1.0-22": "t1zylip4",
    "simclrcfaughead-1.0-33": "7p01aii1",
    "simclrhead-0.05-11": "x1y3cyvr",
    "simclrhead-0.05-22": "x9a09lea",
    "simclrhead-0.05-33": "wrx49m90",
    "simclrhead-0.25-11": "0j3vd0uj",
    "simclrhead-0.25-22": "t48vsxm3",
    "simclrhead-0.25-33": "814m80w1",
    "simclrcfaughead-0.1-11": "7nu2zser",
    "simclrcfaughead-0.1-22": "luxwk9xc",
    "simclrcfaughead-0.1-33": "r5asb39l",
    "simclrcfaughead-1.0-11": "s9ub63yt",
    "supervised-0.05-11": 'yhhb9ytj',
    "supervised-0.05-22": "bx6gi5ib",
    "supervised-0.05-33": 'wbhd0e7x',
    "supervised-0.25-11": 'jtwn5ni9',
    "supervised-0.25-22": 'yll5f08y',
    "supervised-0.25-33": 'mcyuc9hr',
}

In [ ]:
with initialize(version_base=None, config_path="../configs"):
    cfg: DictConfig = compose(
        config_name="config.yaml",
        overrides=["experiment=base_padchestpneumo.yaml", "data.cache=True"],
    )
    print(cfg)
    data_module = PadChestDataModule(config=cfg)

test_dataloader = data_module.test_dataloader()

In [ ]:
df = pd.read_csv(f"../outputs/classification_padchestfinetunepneumo_results.csv")
for run_name, run_id in model_dict_normal.items():
    already_in_df: bool = run_name in df.run_name.values
    if run_id != "" and not already_in_df:
        print(run_name)
        model_to_evaluate = f"../outputs/run_{run_id}/best.ckpt"
        classification_model = ClassificationModule.load_from_checkpoint(
            model_to_evaluate, map_location="cuda:0", strict=False
        ).model.eval()
        classification_model.cuda()

        inference_results = run_inference(test_dataloader, classification_model)
        scanners = inference_results["scanners"]
        for sc in range(2):
            sc_idx = np.where(scanners == sc)
            targets = inference_results["targets"][sc_idx]
            preds = np.argmax(inference_results["confs"], 1)[sc_idx]
            confs = inference_results["confs"][sc_idx][:, 1]
            res = {}
            res["N_test"] = [targets.shape[0]]
            res["Scanner"] = [sc]
            res["run_name"] = run_name
            res["ROC"] = [roc_auc_score(targets, confs)]
            print(res)
            df = pd.concat([df, pd.DataFrame(res, index=[0])], ignore_index=True)

        df.to_csv(
            f"../outputs/classification_padchestfinetunepneumo_results.csv", index=False
        )